<a href="https://colab.research.google.com/github/AzadMehedi/ML-Algorithoms/blob/main/Association_Rules_Apriori_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.filterwarnings('ignore')

In [6]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
df = pd.read_excel(url)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [8]:
df.shape

(541909, 8)

Data cleaning

In [9]:
# cleaning dataset : extra space of 'Description column', dropping null 'InvaoiceNo' rows, dropping rows of transaction credit

# stripping extra space in the Description column
df['Description'] = df['Description'].str.strip()

# dropping the rows without any invoice number
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

# dropping all transactions which were done on credict
df = df[~df['InvoiceNo'].str.contains('C')]

## Now we will separate the data of Italy from the dataset, there will be only parts of InvoiceNo, Description and Quantity.

In [10]:
basket_Italy = df[df['Country'] == 'Italy'].groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
basket_Italy

Description,12 EGG HOUSE PAINTED WOOD,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,16 PIECE CUTLERY SET PANTRY DESIGN,20 DOLLY PEGS RETROSPOT,3 GARDENIA MORRIS BOXED CANDLES,3 ROSE MORRIS BOXED CANDLES,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN RED AND CREAM,...,WOODLAND BUNNIES LOLLY MAKERS,WOODLAND CHARLOTTE BAG,WRAP DOILEY DESIGN,WRAP ENGLISH ROSE,WRAP I LOVE LONDON,WRAP RED APPLES,WRAP RED VINTAGE DOILY,YOU'RE CONFUSING ME METAL SIGN,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
537022,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539752,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
542238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
544399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
545664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
546875,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Now we will process the Italian basket a little more, as a result of which the transaction where a product has been sold more than once will be given 1 otherwise there will be 0.


In [11]:
# converting all positive values into 1 and everything else to 0
def hot_encode(x):
  if x <= 0:
    return 0
  if x >= 1:
    return 1

basket_encoded = basket_Italy.applymap(hot_encode)
basket_Italy = basket_encoded

# Training Model

## Create Association Rules. Let's define support=0.05

In [12]:
# generating frequent itemset
frq_items = apriori(basket_Italy, min_support = 0.05, use_colnames = True) 
  
# generating rules 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules.head(10)   

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
22,(BAKING MOULD CHOCOLATE CUPCAKES),(RED LOVE HEART SHAPE CUP),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf
23,(RED LOVE HEART SHAPE CUP),(BAKING MOULD CHOCOLATE CUPCAKES),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf
54,(HOME BUILDING BLOCK WORD),(BATH BUILDING BLOCK WORD),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf
55,(BATH BUILDING BLOCK WORD),(HOME BUILDING BLOCK WORD),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf
164,(CHRISTMAS CRAFT LITTLE FRIENDS),(CHRISTMAS CRAFT WHITE FAIRY),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf
165,(CHRISTMAS CRAFT WHITE FAIRY),(CHRISTMAS CRAFT LITTLE FRIENDS),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf
166,(CHRISTMAS GINGHAM TREE),(CHRISTMAS GINGHAM STAR),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf
167,(CHRISTMAS GINGHAM STAR),(CHRISTMAS GINGHAM TREE),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf
170,(IVORY WICKER HEART LARGE),(CHRISTMAS GINGHAM STAR),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf
171,(CHRISTMAS GINGHAM STAR),(IVORY WICKER HEART LARGE),0.052632,0.052632,0.052632,1.0,19.0,0.049861,inf
